<a href="https://colab.research.google.com/github/alantypoon/24/blob/main/SyncTalk_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! nvidia-smi

Tue Sep  3 13:55:29 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Install dependencies


In [2]:
! git clone https://github.com/ZiqiaoPeng/SyncTalk.git
%cd SyncTalk

Cloning into 'SyncTalk'...
remote: Enumerating objects: 286, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 286 (delta 105), reused 69 (delta 68), pack-reused 149 (from 1)
Receiving objects: 100% (286/286), 29.52 MiB | 8.48 MiB/s, done.
Resolving deltas: 100% (112/112), done.
/content/SyncTalk


In [ ]:
# about 15 minutes
! apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
! pip install -r requirements.txt
! python ./scripts/install_pytorch3d.py
! pip install ffmpeg-python
# borrow code from GeneFace++
import glob
all_files = glob.glob("*/*.py")
for file in all_files:
  with open(file) as f:
    lines = f.readlines()
    lines = [line.replace('-std=c++14', '-std=c++17') for line in lines]
    if 'backend' in file:
      print(lines)
  with open(file, 'w') as f:
    f.writelines(lines)
! pip install ./freqencoder
! pip install ./shencoder
! pip install ./gridencoder
! pip install ./raymarching

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libasound2-dev is already the newest version (1.2.6.1-1ubuntu1).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 2s (108 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 123597 files and directories currently installed.)
Pre

Looking in links: https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py310_cu121_pyt240/download.html
ERROR: Could not find a version that satisfies the requirement pytorch3d (from versions: none)
ERROR: No matching distribution found for pytorch3d
['import os\n', 'from torch.utils.cpp_extension import load\n', '\n', '_src_path = os.path.dirname(os.path.abspath(__file__))\n', '\n', 'nvcc_flags = [\n', "    '-O3', '-std=c++17',\n", "    '-U__CUDA_NO_HALF_OPERATORS__', '-U__CUDA_NO_HALF_CONVERSIONS__', '-U__CUDA_NO_HALF2_OPERATORS__',\n", ']\n', '\n', 'if os.name == "posix":\n', "    c_flags = ['-O3', '-std=c++17']\n", 'elif os.name == "nt":\n', "    c_flags = ['/O2', '/std:c++17']\n", '\n', '    # find cl.exe\n', '    def find_cl_path():\n', '        import glob\n', '        for edition in ["Enterprise", "Professional", "BuildTools", "Community"]:\n', '            paths = sorted(glob.glob(r"C:\\\\Program Files (x86)\\\\Microsoft Visual Studio\\\\*\\\\%s\\\\VC\\\\Tools\\\\MSVC\\\\

Clone code and download checkpoints

In [ ]:
%cd data
! gdown https://drive.google.com/uc?id=18Q2H612CAReFxBd9kxr-i1dD8U1AUfsV
! unzip May.zip
%cd ../model
! gdown https://drive.google.com/uc?id=1C2639qi9jvhRygYHwPZDGs8pun3po3W7
! unzip trial_may.zip

Inference with target audio

In [ ]:
%cd ..
! python main.py data/May --workspace model/trial_may -O --test --test_train --asr_model ave --portrait --aud ./demo/test.wav

Display output video

In [ ]:
# borrow code from makeittalk
from IPython.display import HTML
from base64 import b64encode
import os, sys
import glob

mp4_name = './model/trial_may/results/ngp_ep0019_audio.mp4'

mp4 = open('{}'.format(mp4_name),'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

print('Display animation: {}'.format(mp4_name), file=sys.stderr)
display(HTML("""
  <video width=256 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url))